# Zomato Daily Report Analysis

#### Objective: This is an exemplary analysis of chat taken by the exicutives to resolve the concern rasied by Zomato application users. 

In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)
pd.set_option('display.min_rows', 20)
pd.set_option('max_colwidth', 50)

In [3]:
zdr = pd.read_excel("C:\\Users\\punit\\Desktop\\Zomato_Daily_Report.xlsx")

In [4]:
# each row represents a single talk
zdr.head()

,convo_id,session_id,assigned_at,resolved_at,chat_for,assigned_to,frt,good_irs,total_irs,lrt,lrt_time,Resolve_Hold,resolution_tag,csat,rated_at,tags,Order_ID,res_name
0,72933357,-Lsl_g7_u-c63FcVT4DH,5/18/2020 0:04,5/18/2020 0:08,India,abhishek.A1016100@jindalx.com,46.0,3,3,3.0,5/18/2020 0:08,Resolve,No Label,NaN,NaN,Order Status,1970202652,The Biryani Life
1,48126604,NaN,5/18/2020 0:04,5/18/2020 0:08,India,sudhir.A1015684@jindalx.com,12.0,5,5,2.0,5/18/2020 0:08,Resolve,No Label,CSAT,5/18/2020 0:08,Packaging issue,1982793617,Subway
2,54656428,NaN,5/18/2020 0:04,5/18/2020 0:08,UAE,Pooja.A1003397@intellicomcenters.com,8.0,9,9,1.0,5/18/2020 0:08,Resolve,No Label,NaN,NaN,NaN,,NaN
3,165621642,NaN,5/18/2020 0:06,5/18/2020 0:12,India,asim.A1013904@intellicomcenters.com,14.0,3,4,2.0,5/18/2020 0:12,Resolve,No Label,CSAT,5/18/2020 0:12,NaN,,NaN
4,125475522,NaN,5/18/2020 0:07,5/18/2020 0:16,India,puneet.A1016939@jindalx.com,14.0,9,9,14.0,5/18/2020 0:16,Resolve,No Label,CSAT,5/18/2020 0:17,"Paytm Refund Issues,Wallet Issue,Wrong Order,O...",1762937125176290073296919495936957218816,"24 Hrs. Tawa V/S Tandoor,24 Hrs. Tawa V/S Tand..."


In [5]:
# data types of all the column, object columns are usually strings, but can also be arbitrary Python objects(lists,dictionaries)
zdr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11554 entries, 0 to 11553
Data columns (total 18 columns):
convo_id          11554 non-null object
session_id        636 non-null object
assigned_at       11554 non-null object
resolved_at       11554 non-null object
chat_for          11554 non-null object
assigned_to       11554 non-null object
frt               10583 non-null float64
good_irs          11554 non-null int64
total_irs         11554 non-null int64
lrt               11431 non-null float64
lrt_time          11431 non-null object
Resolve_Hold      11554 non-null object
resolution_tag    11530 non-null object
csat              4844 non-null object
rated_at          4844 non-null object
tags              9460 non-null object
Order_ID          11554 non-null object
res_name          9465 non-null object
dtypes: float64(2), int64(2), object(14)
memory usage: 1.6+ MB


In [6]:
# count the number of missing values in each column
zdr.isnull().sum()

convo_id              0
session_id        10918
assigned_at           0
resolved_at           0
chat_for              0
assigned_to           0
frt                 971
good_irs              0
total_irs             0
lrt                 123
lrt_time            123
Resolve_Hold          0
resolution_tag       24
csat               6710
rated_at           6710
tags               2094
Order_ID              0
res_name           2089
dtype: int64

## Data Cleaning

In [7]:
# dropping the columns which are not providing any insight for exemplary analysis
zdr.drop(['session_id', 'rated_at', 'res_name'], axis=1, inplace=True)

In [8]:
# droping null values from few columns
zdr.dropna(subset=['frt', 'lrt'], inplace=True)

In [9]:
# changed column name, replaced few long onjects into smaller one, in more readable and easy to analyse style
zdr['csat_dsat'] = zdr['csat'].apply(lambda x: str(x).replace('I want to chat with a support executive', 'DSAT').replace("No, this wasn't helpful", 'DSAT'))
zdr.drop(['csat'], axis=1, inplace=True)

In [10]:
# extracting the extra characters from the rows of assigned_to column
zdr['assigned_to'] = zdr['assigned_to'].str.split('@', n = 1, expand = True)[0]

In [11]:
# converting objects into datetime format
zdr[["assigned_at", "resolved_at", "lrt_time"]] = zdr[["assigned_at", "resolved_at", "lrt_time"]].apply(pd.to_datetime)

In [12]:
# we don't need resolved_at and assigned_at columns but we need difference of these colmuns to get avg handling time(aht)
zdr['aht'] = zdr['resolved_at'] - zdr['assigned_at'] 

In [13]:
# taking out the key content out of each row in aht column
zdr['aht'] = zdr['aht'].dt.components['minutes']

In [14]:
# now we are dropping these columns as we have got what we need
zdr.drop(["assigned_at", "resolved_at", "lrt_time"], axis=1, inplace=True)

In [15]:
# though we don't require Order_ID for our analysis but this column's values are seprated by commas which ultimately form more columns in that row
zdr['Order_ID'] = zdr['Order_ID'].apply(lambda x: str(x).replace(',', ''))
zdr['Order_ID'] = zdr['Order_ID'].map(lambda x: x[0:11])
zdr.drop(['Order_ID'], axis=1, inplace=True)

In [16]:
# converting the data type of few columns
zdr['frt'] = zdr['frt'].astype('int64')
zdr['lrt'] = zdr['lrt'].astype('int64')

In [17]:
# cleaning the tags column
zdr['tags'] = np.where(zdr['tags'].isnull(), False, True)

In [18]:
# for in-depth analysis of this column, we are taking out two columns 
zdr['chat_type'] = zdr['chat_for'].apply(lambda x: str(x).replace('India', 'Food').replace('UAE', 'Food').replace('Lebanon', 'Food'))

In [19]:
zdr['country'] = zdr['chat_for'].apply(lambda x: str(x).replace('Grocery', 'India').replace('UberEats', 'India').replace('callback', 'India'))

In [20]:
# now there is not need of this column as we have extracted the requied data into other columns
zdr.drop(['chat_for'], axis=1, inplace=True)

In [21]:
# irs can be analysed better with it's bifurcation
zdr['irs_bifurcation'] = zdr['good_irs']/zdr['total_irs']

In [22]:
# now we have better parameter to analyse irs
zdr.drop(['good_irs', 'total_irs'], axis=1, inplace=True)

In [23]:
# dropping the special character from the object
zdr['convo_id'] = zdr['convo_id'].apply(lambda x: str(x).replace('-', ''))

In [24]:
# few values are mixed of numbers and alphabets, so we will coerce the values and the replace null values with random value fro the column
random_values = random.randint(2000,2800)
zdr['convo_id'] = pd.to_numeric(zdr['convo_id'], errors='coerce').fillna(random_values).astype('int64')

In [25]:
# renaming all the columns in understandable names
zdr.rename(columns = {'convo_id':'Convo_ID', 'assigned_to':'Assigned_To', 'frt':'FRT', 'irs_bifurcation':'IRS_Bifurcation', 'lrt': 'LRT', 'aht':'AHT', 'csat_dsat':'CSAT_DSAT', 'Resolve_Hold':'Resolve_Hold', 'resolution_tag':'Resolution_Tag', 'tags': 'Cx_Issues_Tag', 'chat_type':'Chat_Type', 'country':'Country' }, inplace = True)

In [26]:
# rearranging the sequence of the columns
zdr = zdr[['Convo_ID', 'Assigned_To', 'FRT', 'IRS_Bifurcation', 'LRT', 'AHT', 'CSAT_DSAT', 'Resolve_Hold', 'Resolution_Tag', 'Cx_Issues_Tag', 'Chat_Type', 'Country']]

In [27]:
# data after cleaning
zdr.head()

,Convo_ID,Assigned_To,FRT,IRS_Bifurcation,LRT,AHT,CSAT_DSAT,Resolve_Hold,Resolution_Tag,Cx_Issues_Tag,Chat_Type,Country
0,72933357,abhishek.A1016100,46,1.00,3,4,nan,Resolve,No Label,True,Food,India
1,48126604,sudhir.A1015684,12,1.00,2,4,CSAT,Resolve,No Label,True,Food,India
2,54656428,Pooja.A1003397,8,1.00,1,4,nan,Resolve,No Label,False,Food,UAE
3,165621642,asim.A1013904,14,0.75,2,6,CSAT,Resolve,No Label,False,Food,India
4,125475522,puneet.A1016939,14,1.00,14,9,CSAT,Resolve,No Label,True,Food,India


1. Convo_ID: ID for conversation between customer and executive
2. Assigned_To: These are the unique ID for all the executives taking the chat with customers
3. FRT: FRT is first response time which means, the very moment when cx arrives at chat and then first reply given to the cx.
4. IRS_Bifurcation: IRS is intermediate response time. Bifercation is of good IRS to total IRS i.e no of replies which are        given in time to customer's query and the total no of replies given to the customer in the whole conversation.
5. LRT: Last response time(LRT). This is the last reply given before the executive resolve the chat.
6. AHT: Average handling time(AHT). AHT helps to give is the time in which sum of all the time taken by each executive to take    the chat to the total no of chat taken.
7. CSAT_DSAT: This parameter is to see the experience given to the customer who come to chat. Customer review by pressing the      customer's satisfaction or dissatisfaction botton. 
8. Resolve_Hold: When executive finish the chat then he/she got two options resolve or hold.
9. Resolution_Tag: This gives us the overview of the chat type.
10. Cx_Issues_Tag: This will tell us the type of the customers issue.
11. Chat_Type: It helps us to know which type of delivery is more entertained by the delivery person.
12. Country: This column helps us to get insight of customer's country.

## Data Analysis

#### 1. Customer who had a chat while logged-in vs offline

In [28]:
cx_offline = (zdr['Convo_ID'] == random_values).sum(axis=0)

In [29]:
cx_logged_in = (zdr['Convo_ID'] != random_values).sum(axis=0)

In [30]:
normalized_value = (cx_offline/cx_logged_in)

In [31]:
print('Number of customers chat while logged-in:', cx_logged_in)
print('Number of customers chat while offline:', cx_offline)
print('Normalized value of customer offline to logged-in:', normalized_value)

Number of customers chat while logged-in: 10506
Number of customers chat while offline: 59
Normalized value of customer offline to logged-in: 0.005615838568437084


#### 2. Executives who are taking less number of chats in whole day

In [32]:
# number of chats taken by executives
chats_per_executive = zdr.Assigned_To.value_counts()

In [33]:
# mean of the total chat taken on the given day
chats_mean = ((zdr.groupby('Assigned_To').size()).mean())/2

In [34]:
# executives who have taken chats less the half of the mean of the day
chats_per_executive[chats_per_executive < chats_mean]

aman.A1013896             19
ashwani.A1016887          19
bulbul.A1007204           19
jahanvi.A1013582          18
sandip.A1015909           18
lakshya.A1010469          17
sachin.A1010829           17
pooja.A1013766            17
sudhir.A1015684           17
rohit.A1011179            16
                          ..
junaid.A1016702            1
sonam.A1007843             1
prince.A1013769            1
inderbakshish.A1014297     1
sukhpreet.A1016919         1
aditya.A1014479            1
mansi.A1016709             1
rahul.A1016715             1
ashish.A1009187            1
trivendra.A1010756         1
Name: Assigned_To, Length: 79, dtype: int64

#### 3. Executive who are defaulters in giving first response to customers on chat

In [35]:
# Executive who have given late frt
zdr.groupby('Assigned_To').FRT.agg(['max']).sort_values('max').tail(20)

,max
Assigned_To,
naveen.A1007643,132
shubham.A1005192,134
Rohit.A1003305,135
durgesh.A1015969,136
nikita.A1016907,137
kshitij.A1006984,139
Ankit.A1016417,141
seema.A1007842,141
kajal.A1014301,143


In [36]:
# Executive who have given lots of late frt, which makes their frt mean high 
zdr.groupby('Assigned_To').FRT.agg(['mean']).sort_values('mean').tail(20)

,mean
Assigned_To,
Akshat.A1003211,30.125000
sachin.A1016583,31.608696
ashwani.A1016887,31.684211
ankush.A1007161,32.090909
nikita.A1016907,32.347222
manoj.A1010821,32.500000
jayant.A1010959,33.375000
ashish.A1014483,33.414634
shivam.A1011517,33.691176


#### 4. Executive who are not giving response in time to customers query

In [37]:
# we are using groupby and agg funtion to get the result
zdr.groupby('Assigned_To').IRS_Bifurcation.agg(['mean']).sort_values('mean').head(20)

,mean
Assigned_To,
aniket.A1014044,0.142857
bharti.A1009821,0.187500
Saurabh.A1002882,0.200000
khalid.A1013953,0.291667
harmeen.A1011269,0.333333
vishnu.A1011617,0.375000
paras.A1013762,0.500000
tarun.A1010832,0.500000
sunil.A1011859,0.557143


#### 5. Executives who are not following the protocols for last response 

In [38]:
# executive whose last response is not as per the protocols
zdr.groupby('Assigned_To').LRT.agg(['mean']).sort_values('mean').head(20)

,mean
Assigned_To,
devendra.A1006825,1.571429
dolly.A1003888,1.571429
kajal.A1014301,1.760870
shivam.A1013788,1.815789
jatin.A1011437,1.883721
junaid.A1016702,2.000000
diwakar.A1007787,2.000000
ashish.A1009187,2.000000
bhawna.A1011045,2.000000


#### 6. Executives who are not entertaining the customers query seriously

In [39]:
# total number of CSAT received by the executives
csat_per_executive = zdr[zdr.CSAT_DSAT=='CSAT'].Assigned_To.value_counts()

In [40]:
# total number of CSAT received by the executives
dsat_per_executive = zdr[zdr.CSAT_DSAT=='DSAT'].Assigned_To.value_counts()

In [41]:
# normalized value of CSAT and DSAT received by the executives
(csat_per_executive/dsat_per_executive).sort_values().dropna().head(20)

shivam.A1014348      0.210526
jitendra.A1012083    0.333333
gitesh.A1007892      0.333333
madhur.A1015741      0.529412
durgesh.A1015969     0.666667
yeshvi.A1016446      0.733333
sachin.A1016583      0.769231
vanshika.A1016923    0.777778
asim.A1013904        0.818182
ashish.A1014483      0.833333
ankita.A1015959      0.866667
pranav.A1015782      0.875000
nitish.A1016496      0.933333
gurleen.A1015736     1.000000
pooja.A1013766       1.000000
latika.raj           1.000000
jahanvi.A1013582     1.000000
jagdish.A1009427     1.000000
prashant.A1016439    1.000000
marshal.A1016365     1.000000
Name: Assigned_To, dtype: float64

#### 7. Some insights of resolution tags

In [42]:
# zdr.groupby('Resolution_Tag').size() 
# alternative method for analysis
zdr.Resolution_Tag.value_counts(normalize=True)

No Label                      0.651853
General Queries               0.139960
Repeat                        0.097508
Groceries India               0.063584
Food not delivered            0.029091
call back support india       0.003601
Merchant Unresponsive         0.003222
Thank you                     0.002938
COD Query                     0.001516
L2 Query                      0.001137
                                ...   
Wrong Merchant Number         0.000284
Gold - Promo-related Query    0.000284
COD Unavailable               0.000190
TR No Query                   0.000190
ID General Pricing            0.000095
Corona Virus Queries          0.000095
TR Cancellation               0.000095
Gold - No Query               0.000095
ID Payment Failure            0.000095
Gold - PG Failure             0.000095
Name: Resolution_Tag, Length: 27, dtype: float64

#### 8. Taking dig into the resolved and hold column

In [43]:
# creating a column which will hold the values in bool
zdr['Resolve_Hold1'] = np.where(zdr['Resolve_Hold']!= 'Resolve', False, True)

In [44]:
# normalized values of resolved and hold chats
zdr.Resolve_Hold1.value_counts(normalize=True)

True     0.995362
False    0.004638
Name: Resolve_Hold1, dtype: float64

In [45]:
# as we won't need this column in further analysis
zdr.drop(['Resolve_Hold1'], axis=1, inplace=True)

#### 9. How many executives are not putting up the customer's concern tags 

In [46]:
zdr.Cx_Issues_Tag.value_counts(normalize=True)

True     0.829437
False    0.170563
Name: Cx_Issues_Tag, dtype: float64

#### 10. Number of issues raised by customer for food vs grocery

In [47]:
zdr.groupby(by='Chat_Type').size()

Chat_Type
Food        9854
Grocery      668
UberEats       4
callback      39
dtype: int64

#### 11. Country from where the customers are raising the issue

In [48]:
zdr.groupby('Country').size()

Country
India      7872
Lebanon     158
UAE        2535
dtype: int64